In [2]:
from hatchet_sdk import new_client
from pydantic import AnyUrl

from epengine.models.shoebox import ShoeboxSimulationSpec

spec = ShoeboxSimulationSpec(
    experiment_id="test",
    sort_index=0,
    lib_uri=AnyUrl("s3://ml-for-bem/tiles/massachusetts/2024_09_30/lib_demo.json"),
    typology="Residential",
    year_built=1972,
    num_floors=3,
    neighbor_polys=["POLYGON ((-10 0, -10 10, -5 10, -5 0, -10 0))"],
    neighbor_floors=[3],
    rotated_rectangle="POLYGON ((5 0, 5 10, 15 10, 15 0, 5 0))",
    rotated_rectangle_area_ratio=1,
    long_edge=10,
    short_edge=10,
    long_edge_angle=0.23,
    footprint_area=100,
    epwzip_path="https://climate.onebuilding.org/WMO_Region_4_North_and_Central_America/USA_United_States_of_America/MA_Massachusetts/USA_MA_Boston-Logan.Intl.AP.725090_TMYx.2009-2023.zip",
)
specs = [spec.model_dump(mode="json")] * 20

client = new_client()

payload = {
    "workflow_name": "simulate_ubem_shoebox",
    "experiment_id": "test-shoebox-recurse",
    "specs": specs,
    "recursion_map": {"factor": 4, "max_depth": 1},
}
client.admin.run_workflow("scatter_gather_recursive", payload)

In [ ]:
selector.BranchesSpec()